## **Regresión Softmax: Explicación Teórica**

La **regresión Softmax** es una generalización de la regresión logística para problemas de clasificación multiclase. Mientras que la regresión logística se usa para clasificar entre dos clases (0 y 1), la regresión Softmax permite clasificar entre múltiples clases modelando probabilidades para cada una de ellas.

---

## **1. Origen: Generalización de la Regresión Logística**

En regresión logística binaria, modelamos la probabilidad de que una instancia $\mathbf{x}$ pertenezca a la clase $y = 1$ usando la función sigmoide:

$$
P(y = 1 | \mathbf{x}) = \frac{1}{1 + \exp(- (\mathbf{w} \cdot \mathbf{x} + b))}
$$

Para clasificación multiclase, necesitamos una función que asigne probabilidades a **múltiples clases** en lugar de solo una. Esto se logra con la **función Softmax**.

---

## **2. Definición de la Función Softmax**

La función Softmax convierte los logits (salidas de la función lineal) en probabilidades:

$$
P(y = k | \mathbf{x}) = \frac{\exp(\mathbf{w}_k \cdot \mathbf{x} + b_k)}{\sum_{j=1}^{K} \exp(\mathbf{w}_j \cdot \mathbf{x} + b_j)}
$$

### **¿Qué significa esta ecuación?**
- Para cada clase $k$, calculamos un **logit** con su propio vector de pesos $\mathbf{w}_k$ y bias $b_k$.
- Aplicamos la función exponencial para mantener las probabilidades **siempre positivas**.
- Normalizamos dividiendo entre la suma de todas las exponenciales, asegurando que las probabilidades sumen 1.

#### **Ejemplo Numérico**
Si tenemos 3 clases y logits calculados como:
$$
z_1 = 2.0, \quad z_2 = 1.0, \quad z_3 = -1.0
$$
Aplicamos Softmax:
$$
P(y=1) = \frac{e^2}{e^2 + e^1 + e^{-1}} = 0.72
$$
$$
P(y=2) = \frac{e^1}{e^2 + e^1 + e^{-1}} = 0.26
$$
$$
P(y=3) = \frac{e^{-1}}{e^2 + e^1 + e^{-1}} = 0.04
$$
Esto indica que la clase 1 es la más probable.

---

## **3. Función de Pérdida: Entropía Cruzada**

Para entrenar el modelo, usamos la **entropía cruzada**:

$$
L = -\sum_{i=1}^{N} \sum_{k=1}^{K} \mathbb{1}(y_i = k) \log P(y_i = k | \mathbf{x}_i)
$$

### **¿Qué significa esta ecuación?**
- Es una suma sobre todas las muestras ($N$) y todas las clases ($K$).
- $\mathbb{1}(y_i = k)$ es 1 si la clase correcta es $k$, y 0 en caso contrario.
- Penaliza predicciones erróneas con mayor intensidad.

---

## **4. Gradiente Descendente: Cómo se Ajustan los Pesos**

Para minimizar la pérdida, calculamos el gradiente de $L$ respecto a los pesos de la clase $k$:

$$
\frac{\partial L}{\partial \mathbf{w}_k} = \sum_{i=1}^{N} \left(P(y_i = k | \mathbf{x}_i) - \mathbb{1}(y_i = k)\right) \mathbf{x}_i
$$

**Actualización de pesos con descenso en gradiente:**
$$
\mathbf{w}_k \leftarrow \mathbf{w}_k - \eta \frac{\partial L}{\partial \mathbf{w}_k}
$$

Donde $\eta$ es la **tasa de aprendizaje**.

### **¿Cómo interpretamos el gradiente?**
- Si la predicción $P(y_i = k)$ es **mayor** que la etiqueta real, reducimos $\mathbf{w}_k$.
- Si es **menor**, aumentamos $\mathbf{w}_k$.

Esto ajusta el modelo para mejorar la clasificación en cada iteración.


In [ ]:
c
# Normalizar los datos
scaler = StandardScaler()
X = scaler.fit_transform(df)

# One-hot encoding de las etiquetas
encoder = OneHotEncoder(sparse=False)
y_one_hot = encoder.fit_transform(y.reshape(-1, 1))

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Inicialización de pesos y bias
np.random.seed(42)
W = np.random.randn(X.shape[1], y_one_hot.shape[1]) * 0.01
b = np.zeros((1, y_one_hot.shape[1]))

# Función Softmax
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Parámetros del entrenamiento
lr = 0.1
num_epochs = 500
loss_history = []

# Gradiente Descendente
for epoch in range(num_epochs):
    logits = np.dot(X_train, W) + b
    y_pred = softmax(logits)
    
    dW = np.dot(X_train.T, (y_pred - y_train)) / len(X_train)
    db = np.sum(y_pred - y_train, axis=0, keepdims=True) / len(X_train)
    
    W -= lr * dW
    b -= lr * db
    
    loss = -np.sum(y_train * np.log(y_pred + 1e-9)) / len(X_train)
    loss_history.append(loss)
    
    if epoch % 50 == 0:
        print(f"Época {epoch}, Pérdida: {loss:.4f}")

# Gráfica de la función de pérdida
plt.plot(loss_history)
plt.xlabel("Épocas")
plt.ylabel("Pérdida")
plt.title("Evolución de la pérdida en Wine Dataset")
plt.show()

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

#### Ahora con sklearn


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd 

# Cargar el dataset Wine
data = load_wine()
X = data.data
y = data.target

# Crear un DataFrame con nombres de columnas
df = pd.DataFrame(X, columns=data.feature_names)
df["target"] = y  # Agregar la columna de la clase


target = 'target'

X = df.drop(target, axis=1)

# Normalizar las características
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

# Crear y entrenar el modelo de Regresión Logística Multiclase (Softmax)
model = linear_model.LogisticRegression(multi_class='multinomial').fit(X_train, y_train)

c:\Users\ppmel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\ppmel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
from sklearn.metrics import accuracy_score

In [8]:
predicciones= model.predict(X_test)

In [10]:
accuracy_score(y_pred=predicciones, y_true=y_test)

0.9550561797752809

In [12]:
# Dataset de coeficientes
coeficientes = pd.DataFrame({'variables': X_train.keys(),
                            'thetas0': model.coef_[0],
                            'thetas1': model.coef_[1],
                            'thetas2': model.coef_[2]})

In [13]:
model.coef_

array([[-2.35408296e-01, -1.49204751e-02,  1.18566719e-01,
        -2.28181754e-01,  1.55565359e-02,  2.29335034e-01,
         4.03904382e-01, -1.82958972e-02,  4.51303208e-03,
        -1.28707826e-01,  1.44853030e-02,  2.06817176e-01,
         6.85629521e-03],
       [ 4.00840309e-01, -5.85380771e-01, -1.36947547e-01,
         1.88109151e-03,  5.35481171e-02,  1.54185498e-01,
         3.52243846e-01, -1.58022517e-02,  3.09327849e-01,
        -1.23089564e+00,  1.92868192e-01,  4.33285700e-01,
        -7.91951642e-03],
       [-1.65432014e-01,  6.00301246e-01,  1.83808276e-02,
         2.26300663e-01, -6.91046529e-02, -3.83520532e-01,
        -7.56148228e-01,  3.40981489e-02, -3.13840881e-01,
         1.35960347e+00, -2.07353495e-01, -6.40102876e-01,
         1.06322121e-03]])

In [14]:
coeficientes

,variables,thetas0,thetas1,thetas2
0,alcohol,-0.235408,0.400840,-0.165432
1,malic_acid,-0.014920,-0.585381,0.600301
2,ash,0.118567,-0.136948,0.018381
3,alcalinity_of_ash,-0.228182,0.001881,0.226301
4,magnesium,0.015557,0.053548,-0.069105
5,total_phenols,0.229335,0.154185,-0.383521
6,flavanoids,0.403904,0.352244,-0.756148
7,nonflavanoid_phenols,-0.018296,-0.015802,0.034098
8,proanthocyanins,0.004513,0.309328,-0.313841
9,color_intensity,-0.128708,-1.230896,1.359603


In [ ]:
z0=(model.coef_[0]* X).sum(axis=1)+model.intercept_[0]

0      4.574868
1      5.158047
2      4.493531
3      6.937278
4      1.023761
         ...   
173   -1.181487
174   -1.369570
175   -0.270735
176   -0.037876
177   -3.430585
Length: 178, dtype: float64

In [18]:
z1=(model.coef_[1]* X).sum(axis=1)+model.intercept_[1]

In [20]:
z2=(model.coef_[2]* X).sum(axis=1)+model.intercept_[2]

In [22]:
np.exp(z0[0])/(np.exp(z0[0])+np.exp(z1[0])+np.exp(z2[0]))

np.float64(0.9895481314687268)

In [23]:
np.exp(z1[0])/(np.exp(z0[0])+np.exp(z1[0])+np.exp(z2[0]))

np.float64(0.010348238068246168)

In [24]:
np.exp(z2[0])/(np.exp(z0[0])+np.exp(z1[0])+np.exp(z2[0]))

np.float64(0.00010363046302689827)